In [28]:
import pandas as pd

In [29]:
customers = pd.read_csv('../Data/Shashi_Kumar_Thakur_Customers.csv')
products = pd.read_csv('../Data/Shashi_Kumar_Thakur_Products.csv')
transactions = pd.read_csv('../Data/Shashi_Kumar_thakur_Transactions.csv')


In [30]:
print(data.columns)


Index(['TransactionID', 'CustomerID', 'ProductID', 'TransactionDate',
       'Quantity', 'TotalValue', 'Price_x', 'CustomerName', 'Region',
       'SignupDate', 'ProductName', 'Category', 'Price_y'],
      dtype='object')


In [31]:
customer_features = data.groupby('CustomerID').agg({
    'TotalValue': 'sum',    
    'Quantity': 'sum',     
    'Price_x': 'mean'      
}).reset_index()

customer_features.rename(columns={'Price_x': 'AvgPrice'}, inplace=True)


print(customer_features.head())


  CustomerID  TotalValue  Quantity    AvgPrice
0      C0001     3354.52        12  278.334000
1      C0002     1862.74        10  208.920000
2      C0003     2725.38        14  195.707500
3      C0004     5354.88        23  240.636250
4      C0005     2034.24         7  291.603333


In [32]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity


scaler = StandardScaler()
features_scaled = scaler.fit_transform(customer_features.iloc[:, 1:])


similarity_matrix = cosine_similarity(features_scaled)
similarity_df = pd.DataFrame(similarity_matrix, index=customer_features['CustomerID'], columns=customer_features['CustomerID'])


In [35]:

lookalike = {}
for cust in customer_features['CustomerID'][:20]:
    similar_custs = similarity_df[cust].sort_values(ascending=False)[1:4]  
    lookalike[cust] = similar_custs.to_dict()


lookalike_df = pd.DataFrame([(k, list(v.keys()), list(v.values())) for k, v in lookalike.items()],
                            columns=['CustomerID', 'SimilarCustomers', 'Scores'])
lookalike_df.to_csv('FirstName_LastName_Lookalike.csv', index=False)
